In [ ]:
from qiskit import QuantumCircuit, transpile, IBMQ
from qiskit.circuit import Delay
from states import Key
from scheme_parameters import SchemeParameters
from encryption_circuit import encrypt
from decryption_circuit import create_decryption_circuit
from deletion_circuit import delete
from attack_circuit import breidbart_measure
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeMontreal
from utils import random_bit_string
import shutil
from datetime import datetime
import experiment
from typing import List, cast
from collections import Counter

In [ ]:
# Experiment parameters
execution_datetime = datetime.now()
# experiment_id = f"cmc-3-{execution_datetime}".replace(":", "-").replace(".", "_")
experiment_id = "oslo-opt-2-again"
system_string = "Fake Montreal" # Or insert other system name here
microsecond_delay = 0 # Delay between preparing the qubits and the first measurement, whether deletion or decryption
folder_prefix = "data"
folder_path = f"{folder_prefix}/{experiment_id}"
circuits_to_save = [] # We will save all the circuits to serialize

# Backend parameters
provider = IBMQ.load_account()

# Ideal simulator
# backend = AerSimulator()

# Simulator with noise model
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)

# Real backend
backend = provider.get_backend("ibm_oslo")

# Qiskit parameters
optimization_level = 2
shots = 1000
qubits_per_circuit = 7

In [ ]:
def run_circuit_blocks(circuits: List[QuantumCircuit]) -> Counter:
    transpiled_circuits = transpile(circuits, backend=backend, optimization_level=optimization_level,
                                    scheduling_method="alap" if microsecond_delay > 0 else None) # type: ignore
    transpiled_circuits = cast(List[QuantumCircuit], transpiled_circuits)
    result = backend.run(transpiled_circuits, shots=shots, memory=True).result()
    shot_list = []
    for shot in range(shots):
        shot_list.append("".join(result.get_memory(i)[shot][::-1] for i in range(len(circuits))))
    circuits_to_save.extend([circuits, transpiled_circuits])
    return Counter(shot_list)

In [ ]:
# Encrypt from new states
scheme_params = SchemeParameters.generate_from_lambda(1)
key = Key.generate_key(scheme_params)
message = random_bit_string(scheme_params.n)
ciphertext = encrypt(message, key, scheme_params, qubits_per_circuit)
if microsecond_delay > 0:
    for circuit in ciphertext.circuits:
        circuit.delay(microsecond_delay, range(circuit.num_qubits), unit="us")
original_circuits = [circuit.copy() for circuit in ciphertext.circuits] # We might modify these circuits in subsequent steps
circuits_to_save.append(original_circuits)

In [ ]:
# Encrypt from existing states
# old_experiment = experiment.Experiment.reconstruct_experiment_from_folder("data/block-encoding-oslo")
# scheme_params = old_experiment.parameters
# key = old_experiment.key
# message = old_experiment.message
# ciphertext = old_experiment.ciphertext
# for circuit in ciphertext.circuits:
#     circuit.data = [instruction for instruction in circuit.data if not isinstance(instruction[0], Delay)]
#     if microsecond_delay > 0:
#         circuit.delay(microsecond_delay, range(circuit.num_qubits), unit="us")
# original_circuits = [circuit.copy() for circuit in ciphertext.circuits] # We might modify these circuits in subsequent steps
# circuits_to_save.append(original_circuits)

In [ ]:
# Test 1 - honest delete
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_circuit_blocks(deletion_circuit_test1)

ciphertext.circuits = [circuit.copy() for circuit in original_circuits]

In [ ]:
# Test 2 - decrypt
decryption_circuit_test2 = create_decryption_circuit(key, ciphertext)
decryption_counts_test2 = run_circuit_blocks(decryption_circuit_test2)

ciphertext.circuits = [circuit.copy() for circuit in original_circuits]

In [ ]:
# Test 3 - honest delete and then decrypt
combined_circuit_test3 = deletion_circuit_test1
# A measurement is always in the computational basis, so measuring again for decryption would be redundant
# The circuits for Test 1 and Test 3 are identical
combined_counts_test3 = deletion_counts_test1

ciphertext.circuits = [circuit.copy() for circuit in original_circuits]

In [ ]:
# Test 4 - malicious delete and then decrypt
breidbart_circuit_test4 = breidbart_measure(ciphertext)
# A measurement is always in the computational basis, so measuring again for decryption would be redundant
combined_counts_test4 = run_circuit_blocks(breidbart_circuit_test4)

ciphertext.circuits = [circuit.copy() for circuit in original_circuits]

In [ ]:
# Test 5 - tamper detection
decryption_circuit_test5 = decryption_counts_test2
combined_counts_test5 = decryption_counts_test2

ciphertext.circuits = [circuit.copy() for circuit in original_circuits]

In [ ]:
# Create experiment and export to a folder
exp = experiment.Experiment(
    experiment_id=experiment_id,
    execution_datetime=execution_datetime,
    execution_shots=shots,
    optimization_level=optimization_level,
    backend_system=system_string,
    qubits_per_circuit=qubits_per_circuit,
    microsecond_delay=microsecond_delay,
    folder_path=folder_path,
    parameters=scheme_params,
    key=key,
    ciphertext=ciphertext,
    message=message,
    deletion_counts_test1=deletion_counts_test1,
    decryption_counts_test2=decryption_counts_test2,
    combined_counts_test3=combined_counts_test3,
    combined_counts_test4=combined_counts_test4,
    combined_counts_test5=combined_counts_test5,
    circuits=circuits_to_save,
)

qubit_list = backend.properties().qubits if backend.properties() else None
exp.export_to_folder(qubit_list=qubit_list)

In [ ]:
# Create zip file
shutil.make_archive("data_export", "zip", folder_prefix);